In [1]:
import numpy as np
import os
import tensorflow as tf
import urllib2
import sys
import matplotlib.pyplot as plt
import cervix_data

sys.path.append('/home/jt2/Workspace/models/slim')
from datasets import imagenet
from datasets import dataset_utils
from nets import inception
from preprocessing import inception_preprocessing
import image_processing
%matplotlib inline

In [2]:
url = "http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz"
checkpoints_dir = '/tmp/checkpoints'

if not tf.gfile.Exists(checkpoints_dir):
    tf.gfile.MakeDirs(checkpoints_dir)

dataset_utils.download_and_uncompress_tarball(url, checkpoints_dir)

>> Downloading inception_v1_2016_08_28.tar.gz 100.0%
Successfully downloaded inception_v1_2016_08_28.tar.gz 24642554 bytes.


In [3]:
slim = tf.contrib.slim
image_size = inception.inception_v1.default_image_size

In [4]:
from preprocessing import inception_preprocessing
import tensorflow as tf

slim = tf.contrib.slim


def load_batch(dataset, batch_size=32, height=299, width=299, is_training=False):
    """Loads a single batch of data.
    
    Args:
      dataset: The dataset to load.
      batch_size: The number of images in the batch.
      height: The size of each image after preprocessing.
      width: The size of each image after preprocessing.
      is_training: Whether or not we're currently training or evaluating.
    
    Returns:
      images: A Tensor of size [batch_size, height, width, 3], image samples that have been preprocessed.
      images_raw: A Tensor of size [batch_size, height, width, 3], image samples that can be used for visualization.
      labels: A Tensor of size [batch_size], whose values range between 0 and dataset.num_classes.
    """
    data_provider = slim.dataset_data_provider.DatasetDataProvider(
        dataset, common_queue_capacity=32,
        common_queue_min=8)
    image_raw, label = data_provider.get(['image', 'label'])
    
    # Preprocess image for usage by Inception.
    image = inception_preprocessing.preprocess_image(image_raw, height, width, is_training=is_training)
    
    # Preprocess the image for display purposes.
    image_raw = tf.expand_dims(image_raw, 0)
    image_raw = tf.image.resize_images(image_raw, [height, width])
    image_raw = tf.squeeze(image_raw)

    # Batch it up.
    images, images_raw, labels = tf.train.batch(
          [image, image_raw, label],
          batch_size=batch_size,
          num_threads=1,
          capacity=2 * batch_size)
    
    return images, images_raw, labels

In [10]:
FLAGS = tf.app.flags.FLAGS
FLAGS.data_dir = '/home/jt2/Workspace/cervix/'
FLAGS.train_dir= '/home/jt2/Workspace/cervix/train'
FLAGS.batch_size = 32
FLAGS.num_readers = 4
FLAGS.num_preprocess_threads = 4
FLAGS.input_queue_memory_factor = 1
FLAGS.image_size = 224

In [6]:
c = cervix_data.CervixData('train')

In [11]:
def get_init_fn():
    """Returns a function run by the chief worker to warm-start the training."""
    checkpoint_exclude_scopes=["InceptionV1/Logits", "InceptionV1/AuxLogits"]
    
    exclusions = [scope.strip() for scope in checkpoint_exclude_scopes]

    variables_to_restore = []
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)

    return slim.assign_from_checkpoint_fn(
      os.path.join(checkpoints_dir, 'inception_v1.ckpt'),
      variables_to_restore)


train_dir = '/tmp/inception_finetuned/'

with tf.Graph().as_default():
    tf.logging.set_verbosity(tf.logging.INFO)
    
#     dataset = flowers.get_split('train', flowers_data_dir)
#     images, _, labels = load_batch(dataset, height=image_size, width=image_size)
    images, labels = image_processing.inputs(c)
    
    # Create the model, use the default arg scope to configure the batch norm parameters.
    with slim.arg_scope(inception.inception_v1_arg_scope()):
#         logits, _ = inception.inception_v1(images, num_classes=c.num_classes(), is_training=True, spatial_squeeze=False)
        logits, _ = inception.inception_v1(images, num_classes=c.num_classes(), is_training=True)
        
    print c.num_classes()
    print images
    print logits
    # Specify the loss function:
    one_hot_labels = slim.one_hot_encoding(labels, c.num_classes())
    print one_hot_labels
    slim.losses.softmax_cross_entropy(logits, one_hot_labels)
    total_loss = slim.losses.get_total_loss()

    # Create some summaries to visualize the training process:
    tf.summary.scalar('losses/Total Loss', total_loss)
  
    # Specify the optimizer and create the train op:
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = slim.learning.create_train_op(total_loss, optimizer)
    
    # Run the training:
    final_loss = slim.learning.train(
        train_op,
        logdir=train_dir,
        init_fn=get_init_fn(),
        number_of_steps=10)
        
  
print('Finished training. Last batch loss %f' % final_loss)

3
Tensor("batch_processing/Reshape:0", shape=(32, 224, 224, 3), dtype=float32, device=/device:CPU:0)
Tensor("InceptionV1/Logits/SpatialSqueeze:0", shape=(32, 3), dtype=float32)
Tensor("OneHotEncoding/one_hot:0", shape=(32, 3), dtype=float32)
Instructions for updating:
Use tf.losses.softmax_cross_entropy instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
Instructions for updating:
Use tf.losses.get_total_loss instead.
Instructions for updating:
Use tf.losses.get_losses instead.
Instructions for updating:
Use tf.losses.get_regularization_losses instead.
INFO:tensorflow:Summary name losses/Total Loss is illegal; using losses/Total_Loss instead.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 11: loss = 1.3791 (22.71 sec/step)
INFO:tensorflow:Stopping Training.
INFO:tensorflow:Finished training! Saving model to disk.
Fini

In [ ]:
slim.one_hot_encoding?